In [1]:
# pip install sentence-transformers

In [2]:
# pip install hdbscan

In [1]:
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer
import hdbscan
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as shc
from google.cloud import bigquery
from sklearn.preprocessing import MultiLabelBinarizer
import datetime as dt
import os
import re
from tqdm import tqdm
warnings.filterwarnings("ignore")

2024-07-18 22:40:07.826378: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-18 22:40:09.132748: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-07-18 22:40:09.132902: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

function to query and process the data and cosine similarity scores

In [2]:
def preprocess_and_get_data(start_date):
    client = bigquery.Client(location="US")

    # query to get the search terms from the trends final report
    query = """
    SELECT search_term, kids, ya, adult, fiction, nonfiction, religious, christian, unknown, lemmatized_search_term, pct_rank
    FROM hc-data-prod-analytics.ds_prod.ds_trends_final_report
    WHERE title_care=False
    AND nonfiction = False
    AND fiction = True
    AND language = 'english'
    AND start_date >= '{}';
    """.format(start_date)
    
    df = client.query(query, location="US").to_dataframe()
    terms = list(df['search_term'].unique())
    
    # query pulls the products associated to the search terms we pulled from the previous query
    def chunks(l, n):
        for i in range(0, len(l), n):
            yield l[i:i + n]

    products = []
    for chunk in chunks(terms, 3000):
        chunk = '("' + '","'.join(chunk) + '")'
        q = """
        SELECT clickeditemname, searchterm 
        FROM `hc-data-prod-analytics.datascience.amazon_weekly_search_term_full`
        WHERE searchterm IN {}
        """.format(chunk)
        products.append(client.query(q, location="US").to_dataframe())
    
    products = pd.concat(products)
    products = products.groupby(['searchterm'])['clickeditemname'].apply(list)
    products = pd.DataFrame(products)
    df = pd.merge(df, products, how='left', left_on='search_term', right_on=products.index)
    
    df = df.reset_index().sort_values(['search_term']).dropna()
    
    # generate embeddings for unique search terms
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    embeddings = model.encode(df['search_term'].unique().tolist())
    embed_df = pd.DataFrame(embeddings, index=df['search_term'].unique())
    
    # transforming the clicked items into a binary matrix
    mlb = MultiLabelBinarizer(sparse_output=True)
    mlb_array = mlb.fit_transform(df['clickeditemname'])
    mlb_array = pd.DataFrame.sparse.from_spmatrix(mlb_array)
    mlb_array['search_term'] = df['search_term'].values
    mlb_array.drop_duplicates(subset=['search_term'], inplace=True)
    mlb_array.index = mlb_array['search_term'].values
    mlb_array.drop('search_term', axis=1, inplace=True)
    
    # merging search term embeddings and clicked items
    sim_df = embed_df.merge(mlb_array, left_on=embed_df.index, right_on=mlb_array.index)
    sim_df.index = sim_df.iloc[:, 0].values
    sim_df.drop(['key_0'], axis=1, inplace=True)
    
    # calculating cosine similarity scores
    sim2 = cosine_similarity(sim_df)
    sims_df = pd.DataFrame(sim2, index=embed_df.index, columns=embed_df.index)
    sims_df.index.name = None
    new_sims_df = sims_df.reset_index().rename(columns={'index': 'Search Term'})
    melt = new_sims_df.melt(id_vars=["Search Term"], var_name="Compared Term", value_name="Similarity Score")
    
    # querying the mapped search terms and trends
    query_trends = """
    SELECT * FROM `hc-data-prod-analytics.oss.bi_dashboard_trends_living_searchterms`
    """
    df_trends = client.query(query_trends, location="US").to_dataframe()
    
    return melt, df_trends

function groups by trend and processess the csv files

In [3]:
def process_trend_data(melt, df_trends, trend, quantile=0.99):
    def dynamic_threshold(df, quantile=0.99):
        thresholds = df.groupby('Search Term')['Similarity Score'].quantile(quantile)
        return thresholds
    
    thresholds = dynamic_threshold(melt, quantile=quantile)
    
    filtered = melt.join(thresholds, on='Search Term', rsuffix='_threshold')
    filtered = filtered[filtered['Similarity Score'] >= filtered['Similarity Score_threshold']].drop('Similarity Score_threshold', axis=1)
    
    # merge cosine sim matrix with trends table
    filtered_with_trends = filtered.merge(df_trends, left_on='Search Term', right_on='searchterm', how='left')
    filtered_with_trends['trend'] = filtered_with_trends['trend'].fillna('Unmapped')
    
    # indicator for unmapped/mapped search terms
    filtered_with_trends['Unmapped/Mapped'] = filtered_with_trends['trend'].apply(lambda x: 'Unmapped' if x == 'Unmapped' else 'Mapped')
    
    # merge compared terms with trends
    compared_trends = df_trends.rename(columns={'searchterm': 'Compared Term', 'trend': 'Compared Trend'})
    filtered_with_trends = filtered_with_trends.merge(compared_trends, on='Compared Term', how='left', suffixes=('', '_compared'))
    filtered_with_trends['Compared Trend'] = filtered_with_trends['Compared Trend'].fillna('Unmapped')
    
    # indicator for unmapped/mapped compared terms
    filtered_with_trends['Compared Term Unmapped/Mapped'] = filtered_with_trends['Compared Trend'].apply(lambda x: 'Unmapped' if x == 'Unmapped' else 'Mapped')
    filtered_with_trends['Final Trend'] = filtered_with_trends.apply(lambda x: x['trend'] if x['trend'] != 'Unmapped' else x['Compared Trend'], axis=1)
    
    # group by final trend
    grouped_peripheral = filtered_with_trends.groupby("Final Trend")
    
    results = []
    for trend_group, group in grouped_peripheral:
        group = group.reset_index(drop=True)
        group = group.sort_values(by="Similarity Score", ascending=False)
        results.append(group)
    
    final_df = pd.concat(results).reset_index(drop=True)
    
    # filter for the specific trend
    if trend in final_df['Final Trend'].values:
        peripheral_df = final_df[final_df['Final Trend'] == trend]
    else:
        peripheral_df = pd.DataFrame(columns=["Search Term", "Compared Term", "Similarity Score", "Final Trend", "Unmapped/Mapped", "Compared Term Unmapped/Mapped"])
        print(f"No peripheral words found for {trend}")

    peripheral_df['Similarity Score'] = peripheral_df['Similarity Score'].round(4)
    # filter out words being compared to itself
    df2 = peripheral_df[peripheral_df['Similarity Score'] < 1]

    # csv file for all similarity scores results for the trend that fall within the threshold
    peripheral_df.to_csv(f'{trend} Similarity Scores.csv', index=False)
    
    # get terms mapped to the trend
    mapped_to_trend = peripheral_df[peripheral_df['Compared Trend'] == trend]['Compared Term'].unique()
    # filter terms not mapped to the trend
    df3 = df2[~df2['Compared Term'].isin(mapped_to_trend) & (df2['Compared Trend'] != 'Unmapped')]
    
    # csv file for compared terms not mapped to og trend
    df3.to_csv(f'{trend} - Compared Terms Not Mapped to Original Trend.csv', index=False)
    
    # csv file for compared terms not mapped to any trend
    df4 = peripheral_df[peripheral_df['Compared Term Unmapped/Mapped'] == 'Unmapped']
    df4.to_csv(f'{trend} - Compared Terms Not Mapped to Any Trend.csv', index=False)

    return peripheral_df, df3, df4

processes the 2 previous functions 

In [4]:
def process_trend(trend, start_date, quantile=0.99):
    melt, df_trends = preprocess_and_get_data(start_date)
    peripheral_df, df3, df4 = process_trend_data(melt, df_trends, trend, quantile)
    return melt, df_trends, peripheral_df, df3, df4

specify the trend you want to look at and the start date for that first initial query that pulls the search terms

In [5]:
%%time

#example
trend = "Enemies to Lovers Romance"
melt, df_trends, peripheral_df, df3, df4 = process_trend(trend, start_date='2024-06-01')

CPU times: user 5min 29s, sys: 35.7 s, total: 6min 5s
Wall time: 1min 42s


In [6]:
print(peripheral_df.shape)
peripheral_df.head()

(900, 9)


,Search Term,Compared Term,Similarity Score,searchterm,trend,Unmapped/Mapped,Compared Trend,Compared Term Unmapped/Mapped,Final Trend
100001,enemies to lovers romance,enemies to lovers romance,1.0,enemies to lovers romance,Enemies to Lovers Romance,Mapped,Romcom Books,Mapped,Enemies to Lovers Romance
100002,enemies to lovers romance,enemies to lovers romance,1.0,enemies to lovers romance,Enemies to Lovers Romance,Mapped,Romantic Comedy,Mapped,Enemies to Lovers Romance
100003,enemies to lovers romance,enemies to lovers romance,1.0,enemies to lovers romance,Enemies to Lovers Romance,Mapped,All Romance,Mapped,Enemies to Lovers Romance
100004,enemies to lovers romance,enemies to lovers romance,1.0,enemies to lovers romance,Enemies to Lovers Romance,Mapped,Enemies to Lovers Romance,Mapped,Enemies to Lovers Romance
100005,enemies to lovers romance books,enemies to lovers romance books,1.0,enemies to lovers romance books,Enemies to Lovers Romance,Mapped,Romcom Books,Mapped,Enemies to Lovers Romance


In [11]:
# looking at the compared terms not mapped to the trend we're looking at
print(df3['Compared Term'].nunique())
# print(df3['Compared Term'].unique())

128


In [12]:
# search terms that are alredy mapped to the trend
print(df4['Search Term'].unique())

['enemies to lovers romance fantasy' 'enemies to lovers free books'
 'enemies to lovers books spicy' 'enemies to lovers dark romance'
 'enemies to lovers romance books' 'enemies to lovers romance']


In [13]:
# unique unmapped terms
print(df4['Compared Term'].nunique())
# print(df4['Compared Term'].unique())

26


In [14]:
# cosine similarity matrix melted to long format
melt.head()

,Search Term,Compared Term,Similarity Score
0,#1 best seller,#1 best seller,1.000000
1,0.00,#1 best seller,0.081819
2,0.00 free kindle books,#1 best seller,0.103107
3,0.00 free kindle books romance,#1 best seller,0.213628
4,0.99,#1 best seller,0.138467


In [15]:
# trends table from `hc-data-prod-analytics.oss.bi_dashboard_trends_living_searchterms`
df_trends.head()

,searchterm,trend
0,tiktok cookbook,Booktok
1,book tok,Booktok
2,booktok books 2024,Booktok
3,tiktok cookbook 2023,Booktok
4,booktok journal,Booktok
